In [5]:
#food-news-media
from bt_candidates.wiring import default_schema_factory, default_filter_factory as f
from bt_candidates.client import Client
from bt_candidates.sorting import SortStrategy
from bt_candidates.common import FieldType, AmountType, MatchType
from datetime import timedelta

CLIENT = Client(host='candidates.aws.boomtrain.com', port=7070)

tf = f.or_filter(
    f.overlap_filter(field='resource-type', values=['article'], min=1),
    f.overlap_filter(field='resource-type', values=['news'], min=1),
)
mf = f.or_filter(
    f.recency_filter(field = 'pubDate', min = timedelta(days=-7)),
    f.recency_filter(field = 'pubDate', min = timedelta(days=-30)),
    f.recency_filter(field = 'pubDate', min = timedelta(days=-90)),
    f.recency_filter(field = 'pubDate', min = timedelta(days=-365))
)
gf = f.named_filter('GLOBAL')

filter = f.and_filter(gf, mf, tf)
candidates = CLIENT.get_candidates('b402ff2a76968f1e8867f52a876690b6', filter=filter, limit=1000, sort_by=SortStrategy.POP_1D)

print(len(candidates))
for c in candidates:
    print(c)

236
Candidate(resource_id='article|qsr-108506', event_counts=None, sort_weight=6)
Candidate(resource_id='article|qsr-108661', event_counts=None, sort_weight=6)
Candidate(resource_id='article|qsr-104011', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-102561', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-108166', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-106276', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-101336', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-104301', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-105326', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-107946', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-103996', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-106351', event_counts=None, sort_weight=5)
Candidate(resource_id='article|qsr-107281', event_counts=Non

In [9]:
import itertools as it

def split_candidates(candidates, needed=10):
    to_score = []
    for _, grp in it.groupby(candidates, lambda c: c.sort_weight):
        grp = list(grp)
        if len(grp) >= needed:
            to_score.extend(grp)
            return to_score
        else:
            to_score.extend(grp)
            
        needed -= len(grp)
    raise ValueError('too few candidates!', needed)
        
split_candidates(candidates, 2)

[Candidate(resource_id='article|qsr-108506', event_counts=None, sort_weight=6),
 Candidate(resource_id='article|qsr-108661', event_counts=None, sort_weight=6)]